In [1]:
# The following lines enable automatic reloading of modules in an IPython/Jupyter environment.
# They work exactly like the commented lines below, but avoid errors when not running in such an environment.
# %load_ext autoreload
# %autoreload 2

try:
    # Only defined inside IPython/Jupyter
    get_ipython().run_line_magic("load_ext", "autoreload")
    get_ipython().run_line_magic("autoreload", "2")
except (NameError, AttributeError):
    # Not running in IPython → just ignore
    pass


In [2]:
# Initializing answer variable
answer = {}


In [3]:
# Some libs that we will use
import torch
import random
import numpy as np
import json_tricks
import lovely_tensors as lt

# Making tensor printouts better
lt.monkey_patch()

# Adding sources to the pythonpath
import sys
root_path = '../../../..'
sys.path.append(root_path)

import dotenv
dotenv.load_dotenv(dotenv.find_dotenv(root_path + '/.env'))

# Importing sources of our project
import src


# Task 0: Prepare the environment

In [ ]:
src.utils.seed.seed_all(0)


# Task 1: Prepare the data

In [4]:
MNIST_train = ...
MNIST_valid = ...
## YOUR CODE HERE


Check that the data is prepared:

In [5]:
train_sample = MNIST_train[0]
valid_sample = MNIST_valid[0]

X_train = train_sample['image']
X_valid = valid_sample['image']
y_train = train_sample['label']
y_valid = valid_sample['label']


In [6]:
## This checks are for dataset verification
answer['X_train.dtype'] = str(X_train.dtype)
answer['y_train.dtype'] = str(y_train.dtype)
answer['X_valid.dtype'] = str(X_valid.dtype)
answer['y_valid.dtype'] = str(y_valid.dtype)
answer['X_train.shape'] = X_train.shape
answer['X_valid.shape'] = X_valid.shape
answer['y_train.shape'] = y_train.shape
answer['y_valid.shape'] = y_valid.shape
answer['X_train.mean'] = float(X_train.mean())
answer['y_train.mean'] = float(y_train.float().mean())
answer['X_valid.mean'] = float(X_valid.mean())
answer['y_valid.mean'] = float(y_valid.float().mean())

print(X_train.dtype, X_valid.dtype, y_train.dtype, y_valid.dtype)
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

print(X_train)
print(X_valid)
print(y_train)
print(y_valid)


In [7]:
import matplotlib.pyplot as plt
plt.imshow(X_train)
plt.show()
print(y_train)


# Task 2: Build the code for the network

This time we will build an autoencoder network that contains two parts:
- Encoder
- Decoder

Your task will be to build a model that contains two of these modules (each Fully-Connected Neural Network) having in mind that volume of Encoder should be exactly equal to the volume of the decoder.

Both encoder and decoder should have the same structure:
```bash
Linear
Activation
Linear     x N
Activation x N
Linear
```

Number of linear and activation layers should be taken from the input list of channels.

For instance, if the list with channels is `[28*28, 128, 64]`, the encoder should look like:
```
28 * 28 -> Linear -> Activation -> 128 -> Linear -> Activation -> 64 -> Linear -> 64
```

The decoder should look like:
```
28 * 28 <- Linear <- 128 <- Activation <- Linear <- 64 <- Activation <- Linear <- 64
```

In this case the image is compressed into a vector of size 64

In [12]:
model = src.models.feedforward.autoencoder.Autoencoder([28 * 28, 128, 64])

src.utils.deterministic_init(model)

check_input = {'image': torch.randn(10, 28 * 28)}
check_output = model(check_input['image'])

answer['check_result'] = src.utils.detach_copy(check_output)


# Task 3: Create loss function and optimizer



In this case we will use Adam with standard parameters as optimizer (`lr=1.0e-4` and weight decay `1.0e-4`)

As a loss function we will use MAE loss:

$L(\mathbf x^*, \mathbf x) = \sum_{s=1}^S |x^*_s - x^s|,$

where index $s$ runs through all the pixels of all images (predicted and input)

In [16]:
import torchmetrics


def loss(pred, targ):
    val = 0
    ## YOUR CODE HERE
    return val

epochs = 10

optimizer = ...

## YOUR CODE HERE


In [17]:
answer['optimizer'] = str(optimizer)

print(optimizer)


# Task 4: Create DataLoaders

In [ ]:
from tqdm import trange, tqdm
import neptune
import os

batch_size = 32
n_epochs = 10

train_dl = ...
valid_dl = ...

## YOUR CODE HERE


# Task 5: Create training loop

Firstly, we will use a service that allows to share the training reports. I strongly recommend using mlflow for your experiments, but for the sake of being able to share the results, we will use [neptune.ai](https://neptune.ai/)

Create an account there, create your first experiment (and name it something like `MNIST FCNN`)

Then create run with `neptune.ai` in your code.

After that you will be able to see on that website the flow of your experiment.

Then it is time to create the training cycle.

Training cycles usually are custom for every network, because of that we will not create one for all the cases.

Training cycles run in epochs each containing 2 stages:
- training
- validation

Here is what you should do in the training stage:
1. extract training batch from training dataloader
2. switch the network to training state (`model.train()`)(will be important for batchnorms and some other modules)
2. generate predictions from the training inputs using the model (`model(inputs)`)
3. calculate the value of the loss that compares the predictions to the targets
4. perform backpropagation (`loss_val.backward()`)
5. make optimization step  with optimizer (`optimizer.step()`)
6. reset optimizer's gradients (`optimizer.zero_grad()`)
7. switch the network to validation state (`model.eval()`)
8. evaluate training the accuracy of your model and update metrics that keep track of accuracy and loss (averaged throughout the training step)

This should be done in iterations for all training batches that come out from `train_dl`

Note that it is extremely important to do `detach` when you memorize accuracy and loss values so that after each iteration RAM is freed automatically by pytorch (because in case we plan to perform backprop, all the intermediate values are needed, and cannot be freed).

Once in a while, if you experience RAM leakage, you should make sure that you detach metrics and loss values that you memorize. In case that does not help, delete all the variables manually and after that call pythons's garbage collector (`gc.collect()`). This will help pytorch with cleaning RAM.

If you do not want to think about detaching and other aspects of graph backpropagation in some part of your code, you may use `torch.no_grad()` context. Exactly that statement we will use during validation stage of the cycle. This context also can be used as a decorator of a function that does not require gradient propagation.

Now it is time to do validation:
- make prediction for validation data
- calculate accuracy for the predictions for both sets
- calculate loss function for the predictions for both sets

Note that the code should look very similarly to training cycle. The only difference is that this time we do not perform backpropagation and optimizer step.

By the end of each epoch, submit the report about accuracies that you gotten and loss values to neptune.ai (run['losses/loss_value/train'].log(train_loss_value))

In [ ]:
def train_model(model, n_epochs, train_dl, valid_dl, loss, optimizer):
    train_loss_history = []
    valid_loss_history = []

    ## YOU SHOULD ADD THE nepune.ai TOKEN BEFORE RUNNING THE TRAINING
    run = neptune.init_run(
        ## YOUR CODE HERE
    )

    for epoch in range(n_epochs):
        train_loss = {'enumerator': 0.0, 'denominator': 1.0e-8}
        valid_loss = {'enumerator': 0.0, 'denominator': 1.0e-8}

        for batch in tqdm(train_dl):
            ...
            ## YOUR TRAINING CODE HERE

        with torch.no_grad():
            for valid_batch in tqdm(valid_dl):
                ...
                ## YOUR EVALUATION CODE HERE

            finalized_train_loss = train_loss['enumerator'] / train_loss['denominator']
            finalized_valid_loss = valid_loss['enumerator'] / valid_loss['denominator']

            # Logging the progress to neptune

            train_loss_history.append(finalized_train_loss)
            valid_loss_history.append(finalized_valid_loss)

    run.stop()
    return train_loss_history, valid_loss_history

train_loss_history, valid_loss_history = train_model(model, n_epochs, train_dl, valid_dl, loss, optimizer)


In [ ]:
answer['train_loss_history'] = train_loss_history
answer['valid_loss_history'] = valid_loss_history

json_tricks.dump(answer, '.answer.json')


# Task 7. Experiment time!

Create a new autoencoder model and train it for 100 epochs (or more if you like)

In [ ]:
n_epochs = 150
model = src.models.feedforward.autoencoder.Autoencoder([28 * 28, 128, 64])
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3, weight_decay=1.0e-8)

train_loss_history, valid_loss_history = train_model(model, n_epochs, train_dl, valid_dl, loss, optimizer)


In [34]:
with torch.no_grad():
    img1 = MNIST_valid[0]['image']
    img2 = MNIST_valid[1]['image']

    plt.figure()
    plt.imshow(img1.reshape([28, 28]))
    plt.figure()
    plt.imshow(img2.reshape([28, 28]))

    emb1 = model.encoder(img1.reshape([1, -1]))
    emb2 = model.encoder(img2.reshape([1, -1]))

    rec1 = model.decoder(emb1)
    rec2 = model.decoder(emb2)

    plt.figure()
    plt.imshow(rec1.reshape([28, 28]))
    plt.figure()
    plt.imshow(rec2.reshape([28, 28]))


In [37]:
with torch.no_grad():
    for a in np.linspace(0, 1, 10):
        plt.figure()
        rec = model.decoder(a * emb1 + (1 - a) * emb2)
        plt.imshow(rec.reshape([28, 28]))
